In [1]:
import os
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [14]:
data_dir = "krylov_data"
all_files = os.listdir(data_dir)
assert "hubbard_exact.h5" in all_files
ev_files = [f for f in all_files if "eigenvalues" in f]
print(len(ev_files))

11


In [18]:
# Get the exact energy.
f_exact = h5py.File(data_dir + "/hubbard_exact.h5", "r")
energy_exact = np.min(f_exact["energies"])
print(f"Exact energy is {energy_exact}.")
f_exact.close()

Exact energy is (1.0717967697244906+0j).


In [25]:
fname = data_dir + "/" + ev_files[0]
# Get the number of steps.
f = h5py.File(fname, "r")
steps = f["steps"][()]
f.close()
# Get the upper bound.
fname2 = fname.replace("eigenvalues", "bound")
f2 = h5py.File(fname2, "r")
# bound = f2["bound"][()]
print(f2.keys())
f2.close()
# Get the dataframe with energy vs. d
df = pd.read_hdf(fname, key="eigenvalues")
df["error"] = abs(df["energy"] - energy_exact) / abs(energy_exact)
df["steps"] = steps
print(df.head())

<KeysViewHDF5 ['gamma0_sq', 'hamiltonian_norm', 'steps']>
   d           eps    energy  num_pos     error  steps
i                                                     
0  3  1.000000e-12  1.072688        3  0.000832    150
1  4  1.000000e-12  1.072114        4  0.000296    150
2  5  1.000000e-12  1.072029        5  0.000216    150
3  6  1.000000e-12  1.071962        6  0.000154    150
4  7  1.000000e-12  1.071920        7  0.000115    150
